# タスクシミュレータ

In [305]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math

## 計画作成

### プロジェクト期間

In [306]:
StartDate = datetime(2021,9 ,1)
EndDate   = datetime(2021,12,31)

### 担当者別係数

In [307]:
owner_coefficient = {
  '田中': 2,
  '山田' : 3.5,
}

### タスク計画

In [308]:
columns_tasks =  ['ID', 'タイトル', '分類', 'タスク種別', '優先度', '順番', '担当者']
tasks = [
  [1234, 'Aを調査する', '残課題', '調査', 1, 2, '田中'],
  [1255, 'Bを調査する', '残課題', '調査', 2, 1, '山田'],
  [1325, 'Cを研究する', 'A要件', '研究', 5, 1, '山田'],
  [1123, 'Dを実装する', 'B要件', '実装', 1, 1, '田中'],
]

df_tasks = pd.DataFrame(data=tasks, columns=columns_tasks)
df_tasks

,ID,タイトル,分類,タスク種別,優先度,順番,担当者
0,1234,Aを調査する,残課題,調査,1,2,田中
1,1255,Bを調査する,残課題,調査,2,1,山田
2,1325,Cを研究する,A要件,研究,5,1,山田
3,1123,Dを実装する,B要件,実装,1,1,田中


### タスク別作業別工数

In [309]:
df_task_detail = df_tasks.copy()

df_task_detail.loc[0, ["基本設計", "詳細設計", "実装"]] = [10, 20, 30]
df_task_detail.loc[1, ["概要作成", "調査", "資料作成"]] = [10, 30, 20]
df_task_detail.loc[2, ["概要作成", "調査", "資料作成"]] = [30, 20, 40]
df_task_detail.loc[3, ["基本設計", "詳細設計", "実装"]] = [15, 30, 20]

df_task_detail.fillna(0, inplace=True)
df_task_detail["基準工数"] = df_task_detail.基本設計 \
                           + df_task_detail.詳細設計 \
                           + df_task_detail.実装 \
                           + df_task_detail.概要作成 \
                           + df_task_detail.調査 \
                           + df_task_detail.資料作成

df_task_detail["工数"] = df_task_detail.apply(lambda x: x.基準工数 * owner_coefficient[x.担当者], axis=1)
df_task_detail



,ID,タイトル,分類,タスク種別,優先度,順番,担当者,基本設計,詳細設計,実装,概要作成,調査,資料作成,基準工数,工数
0,1234,Aを調査する,残課題,調査,1,2,田中,10.0,20.0,30.0,0.0,0.0,0.0,60.0,120.0
1,1255,Bを調査する,残課題,調査,2,1,山田,0.0,0.0,0.0,10.0,30.0,20.0,60.0,210.0
2,1325,Cを研究する,A要件,研究,5,1,山田,0.0,0.0,0.0,30.0,20.0,40.0,90.0,315.0
3,1123,Dを実装する,B要件,実装,1,1,田中,15.0,30.0,20.0,0.0,0.0,0.0,65.0,130.0


### スケジュール

In [310]:
columns_schedules =  ['タイトル', '分類', '担当者', '開始日', '終了日', '単位', '単位あたり工数']
schedules = [
  ['MTG', 'PJ外作業', '田中', StartDate, EndDate, 'w', 5], #週5h
  ['研修', '研修', '山田', datetime(2021,9 ,1), datetime(2021,9 ,1), 'd', 5]
]
df_schedules = pd.DataFrame(data=schedules, columns=columns_schedules)
df_schedules['開始日'] = pd.to_datetime(df_schedules['開始日'])
df_schedules['終了日'] = pd.to_datetime(df_schedules['終了日'])
df_schedules

,タイトル,分類,担当者,開始日,終了日,単位,単位あたり工数
0,MTG,PJ外作業,田中,2021-09-01,2021-12-31,w,5
1,研修,研修,山田,2021-09-01,2021-09-01,d,5


## シミュレート

In [311]:
format = "%Y/%m/%d"
f'PJ期間: {StartDate.strftime(format)} ～ {EndDate.strftime(format)}'

'PJ期間: 2021/09/01 ～ 2021/12/31'

In [312]:
days_per_month = 20 #月あたり稼働日数
weeks_per_month = 4 #月当たり週数

def calculate_days(x):
  from_date = x.開始日.to_pydatetime() if x.開始日.to_pydatetime() > m_start else m_start
  to_date = x.終了日.to_pydatetime() if x.終了日.to_pydatetime() < m_end else m_end
  if x.単位 == 'd':
    return (to_date - from_date).days + 1
  elif x.単位 == 'w':
    return math.floor((to_date - from_date).days / 7)
  elif x.単位 == 'm':
    return 1

### 月別スケジュール別工数

In [313]:
m_start = StartDate
dfs = []
while(True):
  m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
  schedule_m = df_schedules.query(f"開始日 <= '{m_end}' and 終了日 >= '{m_start}'", engine='python').copy()
  schedule_m["月"] = m_start
  schedule_m["単位数"] = schedule_m.apply(lambda x: calculate_days(x), axis=1)
  schedule_m["月あたり予定工数"] = schedule_m["単位あたり工数"] * schedule_m["単位数"]
  dfs.append(schedule_m.copy())

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

schedule_bym = pd.concat(dfs, axis=0)
schedule_bym

,タイトル,分類,担当者,開始日,終了日,単位,単位あたり工数,月,単位数,月あたり予定工数
0,MTG,PJ外作業,田中,2021-09-01,2021-12-31,w,5,2021-09-01,4,20
1,研修,研修,山田,2021-09-01,2021-09-01,d,5,2021-09-01,1,5
0,MTG,PJ外作業,田中,2021-09-01,2021-12-31,w,5,2021-10-01,4,20
0,MTG,PJ外作業,田中,2021-09-01,2021-12-31,w,5,2021-11-01,4,20
0,MTG,PJ外作業,田中,2021-09-01,2021-12-31,w,5,2021-12-01,4,20


### 月別担当者別予定時間

In [314]:
schedule_bym_owner = schedule_bym.groupby(['月','担当者']).月あたり予定工数.sum().reset_index().copy()
schedule_bym_owner

,月,担当者,月あたり予定工数
0,2021-09-01,山田,5
1,2021-09-01,田中,20
2,2021-10-01,田中,20
3,2021-11-01,田中,20
4,2021-12-01,田中,20


In [315]:
hours_per_day = 6.5 #1日あたり稼働時間
days_per_week = 5
buffer_m = 0.2 #月ごとのバッファ
datetime_format = '%Y-%m-%d %H:%M:%S'
m_start = StartDate
# df_unassigned = df_task_detail.copy()
dfs_freetime = []
while(True):
  for owner in owner_coefficient:
    m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
    to_date = EndDate if EndDate < m_end else m_end
    weeks = math.floor((to_date - m_start).days / 7)
    
    # 月の空き時間計算
    all_time = hours_per_day * weeks * days_per_week
    buffer_time = all_time * buffer_m
    schedule_time = schedule_bym_owner.query(f"担当者=='{owner}' and 月=='{m_start}'").月あたり予定工数.sum()
    free_time = all_time - buffer_time - schedule_time
    
    dfs_freetime.append([m_start, owner, free_time, all_time, schedule_time, buffer_time])

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

df_freetime = pd.DataFrame(data=dfs_freetime, columns=["月","担当者","空き時間","月総時間","予定時間","バッファ"])
df_freetime

,月,担当者,空き時間,月総時間,予定時間,バッファ
0,2021-09-01,田中,84.0,130.0,20,26.0
1,2021-09-01,山田,99.0,130.0,5,26.0
2,2021-10-01,田中,84.0,130.0,20,26.0
3,2021-10-01,山田,104.0,130.0,0,26.0
4,2021-11-01,田中,84.0,130.0,20,26.0
5,2021-11-01,山田,104.0,130.0,0,26.0
6,2021-12-01,田中,84.0,130.0,20,26.0
7,2021-12-01,山田,104.0,130.0,0,26.0


### 月別担当者別タスク

In [316]:
hours_per_day = 6.5 #1日あたり稼働時間
days_per_week = 5
buffer_m = 0.2 #月ごとのバッファ
datetime_format = '%Y-%m-%d %H:%M:%S'
m_start = StartDate
df_unassigned = df_task_detail.copy()
df_unassigned["月工数"] = df_unassigned.工数
dfs_sumulate = []
while(True):
  for owner in owner_coefficient:
    m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
    
    # 月の空き時間計算
    free_time = df_freetime.query(f"担当者=='{owner}' and 月=='{m_start}'").空き時間.sum()
    
    # 未割当タスク
    df_unassinge_owner = df_unassigned.query(f"担当者=='{owner}'").sort_values(["優先度","順番"]).copy()
    df_unassinge_owner["累積工数"] = df_unassinge_owner.月工数.cumsum()

    # 累積工数内のタスクをアサインする
    df_assign_owner = df_unassinge_owner.query(f"累積工数<={free_time}").copy()
    df_assign_owner["月"] = m_start.replace(day=1)
    dfs_sumulate.append(df_assign_owner.copy())
    df_unassigned.drop(df_assign_owner.index, inplace=True)
    
    # 空き時間がでたら残工数内で対応
    m_summary = df_assign_owner.月工数.sum()
    df_unassinge_owner = df_unassigned.query(f"担当者=='{owner}'").sort_values(["優先度","順番"]).copy()
    if m_summary < free_time and len(df_unassinge_owner) > 0:
      devided_time = free_time - m_summary
      df_devide = df_unassinge_owner.sort_values(["優先度","順番"]).head(1)
      # 工数を減らす
      df_unassigned.loc[df_devide.index,"月工数"] = df_devide.月工数 - devided_time
      # あまり工数でタスクを分割する
      df_assign_owner_task = df_devide.copy()
      df_assign_owner_task["月"] = m_start
      df_assign_owner_task["月工数"] = devided_time
      dfs_sumulate.append(df_assign_owner_task.copy())

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

df_simulate = pd.concat(dfs_sumulate, axis=0)

In [317]:
df_simulate.sort_values(['月','優先度','順番'])[['ID', 'タイトル', '分類', 'タスク種別', '優先度', '順番', '担当者','工数','月','月工数']]

,ID,タイトル,分類,タスク種別,優先度,順番,担当者,工数,月,月工数
3,1123,Dを実装する,B要件,実装,1,1,田中,130.0,2021-09-01,84.0
1,1255,Bを調査する,残課題,調査,2,1,山田,210.0,2021-09-01,99.0
3,1123,Dを実装する,B要件,実装,1,1,田中,130.0,2021-10-01,46.0
0,1234,Aを調査する,残課題,調査,1,2,田中,120.0,2021-10-01,38.0
1,1255,Bを調査する,残課題,調査,2,1,山田,210.0,2021-10-01,104.0
0,1234,Aを調査する,残課題,調査,1,2,田中,120.0,2021-11-01,82.0
1,1255,Bを調査する,残課題,調査,2,1,山田,210.0,2021-11-01,7.0
2,1325,Cを研究する,A要件,研究,5,1,山田,315.0,2021-11-01,97.0
2,1325,Cを研究する,A要件,研究,5,1,山田,315.0,2021-12-01,104.0


### タスク別期間

In [325]:
df_term = df_simulate.groupby('ID').月.agg({'min','max'}).reset_index()
df_term.rename(columns={'min': '開始月', 'max': '終了月'}, inplace=True)
pd.merge(df_task_detail, df_term, on='ID', how='left')[['ID', 'タイトル', '分類', 'タスク種別', '優先度', '順番', '担当者','工数','開始月','終了月']]

,ID,タイトル,分類,タスク種別,優先度,順番,担当者,工数,開始月,終了月
0,1234,Aを調査する,残課題,調査,1,2,田中,120.0,2021-10-01,2021-11-01
1,1255,Bを調査する,残課題,調査,2,1,山田,210.0,2021-09-01,2021-11-01
2,1325,Cを研究する,A要件,研究,5,1,山田,315.0,2021-11-01,2021-12-01
3,1123,Dを実装する,B要件,実装,1,1,田中,130.0,2021-09-01,2021-10-01
